# Databricks Storage Mounts

**Benefits**
- Access data without requiring credentials
- Access files using file semantics instead of storage URLs
- Stores files to object storage realizing Azure storage benefits

**Workflow**
- Mounts ADLS Gen2 to DBFS Using Service Principal
- Access DBFS from Databricks Notebooks

In [0]:
# Set Variables

client_id = ""
tenant_id = ""
secret_value = ""

In [0]:
# Set Variables Using Secret Scope

client_id = dbutils.secrets.get(scope=<secret_scope>, key=<secret_scope_key>)
tenant_id = dbutils.secrets.get(scope=<secret_scope>, key=<secret_scope_key>)
secret_value = dbutils.secrets.get(scope=<secret_scope>, key=<secret_scope_key>)
service_credential = dbutils.secrets.get(scope="<secret_scope>",key="<service_scope_key>")

In [0]:
# Set Spark Config

spark.conf.set("fs.azure.account.auth.type.<storage_account_namet>.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.<storage_account_namet>.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.<storage_account_namet>.dfs.core.windows.net", "<application_id>")
spark.conf.set("fs.azure.account.oauth2.client.secret.<storage_account_name>.dfs.core.windows.net", <service_credential_client_secret>)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.<storage_account_namet>.dfs.core.windows.net", "https://login.microsoftonline.com/<directory_id>/oauth2/token")

### Additional Configuration Code

In [0]:
# Databricks Notebook Variables

storage_account_name = "formula1dl"
client_id = dbutils.secrets.get(scope="formula1-scope", key="databricks-app-client-id")
tenant_id = dbutils.secrets.get(scope="formula1-scope", key="databricks-app-tenant-id")
client_secret = dbutils.secrets.get(scope="formula1-scope", key="databricks-app-client-secret")

In [0]:
# Set Configurations

configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": f"{client_id}",
    "fs.azure.account.oauth2.client.secret": f"{client_secret}",
    "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
}

In [0]:
# Create Mount Function

def mount_adls(container_name):
  dbutils.fs.mount(
    source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
    mount_point = f"/mnt/{storage_account_name}/{container_name}",
    extra_configs = configs
  )

In [0]:
# Mount Containers

mount_adls("raw")
mount_adls("processed")
mount_adls("presentation")

In [0]:
# Display Contents

dbutils.fs.ls("/mnt/formula1dl/raw")
dbutils.fs.ls("/mnt/formula1dl/processed")
dbutils.fs.ls("/mnt/formula1dl/presentation")

In [0]:
# Unmount Containers

dbutils.fs.unmount("/mnt/formula1dl/raw")
dbutils.fs.unmount("/mnt/formula1dl/processed")
dbutils.fs.unmount("/mnt/formula1dl/presentation")